# Book Categorization Using Bayesian Classification

## Introduction

![Python Logo](Screenshot%202023-10-22%20at%208.30.57%20PM.png)


#### Mohammad Taha Majlesi 810101504
## probebility and statics project 1

baysian classifier for categorizing books
for running this project it takes over 10 minutes to run all of cells
but i write main results in cells and you can run them and see results






## Readind Training Date from CSV
first we need to read data from csv train dataset and show it for better understanding




In [ ]:
import pandas as pd
Train_data_frame = pd.read_csv('books_train.csv')
Train_data_frame


***we make new dataframe that name is category_data_frame***
we need to make new dataframe that has 3 columns
1. categories
2. data
3. BOW
and then we need to put all of data in one row for each category


that data frame name is category_data_frame
we add all of description and title of each category in one row
by that we have 6 rows 



In [ ]:
category_data_frame = pd.DataFrame(columns=['categories','data','BOW'])

category_data_frame['categories'] = Train_data_frame['categories'].unique()

category_data_frame['data'] = ' '.join(Train_data_frame['description'])
for i in category_data_frame['categories']:
    category_data_frame['data'][category_data_frame['categories'] == i] = ' '.join(Train_data_frame['description'][Train_data_frame['categories'] == i] + ' ' + Train_data_frame['title'][Train_data_frame['categories'] == i])


category_data_frame







## phase 1
# Preprocessing Data

we need function for removing numbers and special characters and then we need to tokenize data
we use HAZM library for that and regex library for removing special characters and numbers
and then we need to tokenize that data

for step words we use hazm library step words and then we add more step words to that list
and additional step words are in json file we add that to list and then we use that list for removing step words from data


In [ ]:
json_step_words = pd.read_json('step_words.json')
print(len(json_step_words) )
more_step_words = list(json_step_words['stopWords'])


__we have tow function for preprocessing data one with step words and one without step words for comparing results__
useing data = re.sub(r'[^\w\s]','',data) is for removing special characters and useing data = re.sub(r'\d+','',data) is for removing numbers


In [ ]:
from hazm import *
import re

def preprocess_with_step_word(data):
    step_words = stopwords_list()
    step_words += more_step_words

    normalizer = Normalizer()
    # normalizing data
    data = normalizer.normalize(data)

    # removing special characters
    data = re.sub(r'[^\w\s]','',data)
    
    # numbers and /n removing
    data = re.sub(r'\d+','',data)
    data = re.sub(r'\n+',' ',data)
    data = word_tokenize(data)
    data = [item for item in data if item not in step_words]
    # data = [Stemmer().stem(item) for item in data]
    
    return data

now we preprocess data 

In [ ]:
category_data_frame['data'] = category_data_frame['data'].apply(lambda x: preprocess_with_step_word(x))
' '.join(category_data_frame['data'][0])

## show data and plot it after normalizing and removing special characters and numbers and removing step words

In [ ]:


import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.bar(category_data_frame['categories'],[len(category_data_frame['data'][i]) for i in range(len(category_data_frame['data']))])


### show data numbers 

In [ ]:
for i in range(len(category_data_frame['data'])):
    print(category_data_frame['categories'][i],len(category_data_frame['data'][i]))
    


## MAKE BAG OF WORDS
we need to make bag of words for each category and assign each word to a number


### Making Unique Words of all of words in trained data set

In [ ]:
#dataframe for unique words
unique_words = pd.DataFrame(columns=['words'])
all_train_data = ( ' '.join([i for i in Train_data_frame['categories']]))
all_train_data = all_train_data + ' '.join([i for i in Train_data_frame['title']])
all_train_data = preprocess_with_step_word(all_train_data)
all_train_data = list(set(all_train_data))
unique_words['words'] = all_train_data
len(unique_words)



__this show that we have around 4000 unique words in all of data set__

## Make Bag of Words with unique words
for all of categories we need to calculate how many times each word is in each category
در اینجا ما یک ارایه از اعداد که ظرفیت ۵۰۰۰۰ تایی دارند را به قسمت BOW در دیتا فریم اضافه میکنیم 



In [ ]:

for i in range(len(category_data_frame['categories'])):
    BOW = [0] * len(unique_words)


    for index, word in enumerate(unique_words['words']):
        if word in category_data_frame['data'][i]:
            count = category_data_frame['data'][i].count(word)
            BOW[index] += count
    category_data_frame['BOW'][i] = BOW




## make plot of words in each category for 1 - 1000

صرفا برای نشان دادن تجمع کلمات در هر کدام از دسته ها 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.bar(unique_words['words'][:1000],category_data_frame['BOW'][0][:1000])
plt.show()

plt.figure(figsize=(20,10))
plt.bar(unique_words['words'][:1000],category_data_frame['BOW'][1][:1000])
plt.show()

plt.figure(figsize=(20,10))
plt.bar(unique_words['words'][:1000],category_data_frame['BOW'][2][:1000])
plt.show()

plt.figure(figsize=(20,10))
plt.bar(unique_words['words'][:1000],category_data_frame['BOW'][3][:1000])
plt.show()

plt.figure(figsize=(20,10))
plt.bar(unique_words['words'][:1000],category_data_frame['BOW'][4][:1000])
plt.show()

plt.figure(figsize=(20,10))
plt.bar(unique_words['words'][:1000],category_data_frame['BOW'][5][:1000])
plt.show()




## Phase 2

## Making Prediction Function
we need to make prediction function for each word in each category
we use Bayes rule for that
P(C|X) = P(X|C) * P(C) / P(X)
P(C|X) = probability of category C when we have X
P(X|C) = probability of X when we have category C
P(C) = probability of category C
P(X) = probability of X
we need to calculate P(X|C) for each word in each category
and then we need to calculate P(C) for each category
but we know that p(C) is same 1/6 for each category
so we dont need to calculate that
and because of that we want to compare P(X|C) for each category and then we choose category that has higher probability we dont need to calculate P(X) because it is same for each category
and just we need to calculate P(X|C) for each category
and then we need to multiply all of P(xi|C) for each word in each category
because we have more than one word in each example and probebility of each word is independent from other words
so we need to multiply all of P(xi|C) for each word in each category
for calculate P(X|C) we need to calculate P(xi|C) for each word in each category
P(X|C) = P(x1|C) * P(x2|C) * P(x3|C) * ... * P(xn|C)
and then we use log for that because we have very small numbers and we have underflow
so we use log for that
log(P(X|C)) = log(P(x1|C)) + log(P(x2|C)) + log(P(x3|C)) + ... + log(P(xn|C))


if we do not have a word in a category we have 0 for that word in that category and it is not correct
so we need to use additive smoothing for that
we add 1 to each word in each category
so we have 1 for each word in each category


## making prediction function for one word in one category 


٢‐ فرض کنید در شرایطی متن توضیحات یک کتاب طولانی باشد در این صورت با ضرب شدن احتمالهای هر کلمه چه
اتفاقی می افتد؟ پیشنهاد شما برای رفع این مشکل چیست؟

همان گونه که گفته شد از لوگاریتم استفاده میکنیم یعنی لوگاریتم ها را باید با هم جمع کنیم 
در تابع پایین خروجی لوگاریتم است 


در این قسمت ما تابعی برای احتمال وجود هر کلمه داریم 
چون که احتمال وجود هر کلمه از تقسیم کردن مقدار موجود در بگ او ورد بر تعداد کل کلمات است پس اگر به طور عادی بخواهیم برای هر کدارم از کلمات این مقدارهارا در هم ضرب کنیم عددی بسیار کوچک داریم .
بنابر این از تابع لوگاریتم استفاده میکنیم .
یعنی باید برای تک تک کلمات لوگاریتم های ان هارا با هم جمع کنیم .


دو تا تابع برای پیشبینی داریم یکی با اسم additive smoothing که اگر کلمه ای در دسته ای وجود نداشته باشد به جای صفر یک میگذارد و یک تابع دیگر بدون اسم که اگر کلمه ای در دسته ای وجود نداشته باشد صفر را بر میگرداند .


if we have a word that is not in unique words we have 0 for that word in each category and we dont have any prediction for that word in each category

In [ ]:
from math import log
def predict_without_additive_smooth(word,category):
    if word not in list(unique_words['words']):
        return 0
    index = unique_words[unique_words['words'] == word].index[0]
    if category_data_frame['BOW'][category][index] == 0:
        return 0
    return log(category_data_frame['BOW'][category][index] / sum(category_data_frame['BOW'][category]))


def predict_with_additive_smooth(word,category):
    if word not in list(unique_words['words']):
        return 0
    index = unique_words[unique_words['words'] == word].index[0]
    if category_data_frame['BOW'][category][index] == 0:
        return log(1/(sum(category_data_frame['BOW'][category])+1))
    return log(category_data_frame['BOW'][category][index]/sum(category_data_frame['BOW'][category]))





## test prediction function
this be negative prediction JUST simple test


In [ ]:
predict_with_additive_smooth('کتاب',0)

for predicting one sentence we need to sum all of words in that sentence

In [ ]:
sample1 = """«قلعه‌ی حیوانات» جورج اورول، درباره گروهی از حیوانات اهلی است که در اقدامی آرمان‌گرایانه و انقلابی، صاحب مزرعه (آقای جونز) را از مزرعه‌اش فراری می‌دهند تا خودشان اداره مزرعه را به‌دست گرفته و «برابری» و «رفاه» را در جامعه خود برقرار سازند. رهبری این جنبش را گروهی از خوک‌ها به‌دست دارند ولی پس از مدتی این گروه جدید نیز به رهبری خوکی به نام ناپلئون، به بهره‌کشی از حیوانات مزرعه می‌پردازند و هرگونه مخالفتی را سرکوب می‌کنند.
اورول به عنوان یک سوسیال- دموکرات در جریان جنگ داخلی اسپانیا با سیاست‌های حکومت سوسیالیستی شوروی آشنا شده‌بود و از پاکسازی‌های خشونت‌آمیز دوران استالین خشمگین بود. او با نگارش این رمان از استبداد طبقه حاکم شوروی به‌سختی انتقاد کرد و معتقد بود نظام شوروی تبدیل به دیکتاتوری شده و برپایه کیش شخصیت بنا گشته‌است. در این رمان انقلاب حیوانات مزرعه، نماد انقلاب کارگری و سرنوشت آن بر ضد نظام سرمایه‌داری است.
بخشی از رمان: 
یکی از یک‌شنبه‌ها، صبح که حیوانات برای گرفتن دستور جمع شده بودند، ناپلئون اعلام داشت که سیاست جدیدی اتخاذ کرده است: از این تاریخ به بعد مزرعه‌ی حیوانات با مزارع مجاور دادوستد خواهد کرد، البته نه به منظور تجارت، بلکه برای به دست‌آوردن مواد مورد نیاز. گفت که آسیاب بادی باید بر هر چیز دیگری مقدم باشد. فعلاً مقداری یونجه و مقداری گندم فروخته خواهد شد و بعد اگر به پول بیش‌تری نیاز باشد از طریق فروش تخم‌مرغ، که همیشه در ولینگدن بازار دارد تامین خواهد شد. ناپلئون اضافه کرد که مرغ‌ها باید از فداکاری که به منظور کمک و شرکت در امر ساختمان آسیاب بادی ست استقبال کنند. یک بار دیگر حیوانات به شکلِ مبهمی احساس ناراحتی کردند. ارتباط نداشتن با بشر، معامله‌ی تجاری‌نکردن، پول به کار نبردن، مگر این‌ها جزو تصمیمات نخستین نشست پس از اخراج آقای جونز نبود؟ همه‌ی حیوانات این تصمیمات نخستین را به خاطر داشتند و یا لااقل تصور می‌کردند آن‌ها را به خاطر دارند."""

category_name = 'رمان'

sample = preprocess_with_step_word(sample1)
print(sample)


In [ ]:
all_predictions = []

ss= 0
for i in range(6):
    prediction = 0
    for word in sample:
        prediction += predict_without_additive_smooth(word,i)
    all_predictions.append(prediction)
    
print(all_predictions)
print(all_predictions.index(max(all_predictions)))

all_predictions = []

ss= 0
for i in range(6):
    prediction = 0
    for word in sample:
        prediction += predict_with_additive_smooth(word,i)
    all_predictions.append(prediction)
    
print(all_predictions)
print(all_predictions.index(max(all_predictions)))


that you see last has lower prediction so it is our prediction
prediction with additive smoothing is better and correct


## Prediction function for all of words in one example and all of categories
this fuction return prediction of each category in list

In [ ]:
def predict_all_without_additive_smooth(sample):
    all_predictions = []
    for i in range(6):    
        prediction = 0
        for word in sample:
            prediction += predict_without_additive_smooth(word,i)
        all_predictions.append(prediction)
    return all_predictions



def predict_all_with_additive_smooth(sample):
    all_predictions = []
    for i in range(6):    
        prediction = 0
        for word in sample:
            prediction += predict_with_additive_smooth(word,i)
        all_predictions.append(prediction)
    return all_predictions




In [ ]:
print(predict_all_without_additive_smooth(sample))
print(predict_all_with_additive_smooth(sample))


## fucntion for get category name by prediction of sentence

In [ ]:
def get_category_name_by_prediction_with_additive_smooth(data):
    data = preprocess_with_step_word(data)
    predictions = predict_all_with_additive_smooth(data)
    return category_data_frame['categories'][predictions.index(max(predictions))]

def get_category_name_by_prediction_without_additive_smooth(data):
    data = preprocess_with_step_word(data)
    predictions = predict_all_without_additive_smooth(data)
    return category_data_frame['categories'][predictions.index(max(predictions))]


In [ ]:

get_category_name_by_prediction_with_additive_smooth(sample1)

## plot function of prediction of each category for one example

In [ ]:
def plot_prediction(data):
    data = preprocess_with_step_word(data)
    predictions = predict_with_additive_smooth(data)
    plt.figure(figsize=(20,10))
    plt.bar(category_data_frame['categories'],predictions)
    plt.show()


## Get percentage of samples that predicted correctly
reading samples data
for all of three types of predictions

In [ ]:
sample_data = pd.read_csv('books_test.csv')
def predict_result_with_additive_smooth(data):
    list_of_preds = []
    for i in range(450):
         list_of_preds.append([sample_data['categories'][i],get_category_name_by_prediction_with_additive_smooth(sample_data['description'][i]+' '+sample_data['title'][i])])
    corrects = 0
    for i in list_of_preds:
        if i[0] == i[1]:
            corrects += 1
    return corrects/450

def predict_result_without_additive_smooth(data):
    list_of_preds = []
    for i in range(450):
         list_of_preds.append([sample_data['categories'][i],get_category_name_by_prediction_without_additive_smooth(sample_data['description'][i]+' '+sample_data['title'][i])])
    corrects = 0
    for i in list_of_preds:
        if i[0] == i[1]:
            corrects += 1
    return corrects/450



## now we can get percent of correct prediction for each type of prediction

In [ ]:
predict_result_with_additive_smooth_res = predict_result_with_additive_smooth(sample_data)
print("percent of correct prediction with additive smooth and step word is : ",predict_result_with_additive_smooth_res)

predict_result_without_additive_smooth_res = predict_result_without_additive_smooth(sample_data)
print("percent of correct prediction without additive smooth and step word is : ",predict_result_without_additive_smooth_res)

## plot Result of prediction for each type of prediction

In [ ]:
plt.figure(figsize=(20,10))
plt.bar(['with additive smooth and step word','without additive smooth and step word'],[predict_result_with_additive_smooth_res,predict_result_without_additive_smooth_res])
plt.show()

## we get 0.8288888888888889  percent with additive_smooth

difrence between phase 2 and phase 3 in unique words
change unique words for stemmer phase 3

## Phase 3 stemming  and minimizing 

## Stemmer

In [ ]:
main_unique_words = unique_words.copy()  #for making copy 

In [ ]:
unique_words = main_unique_words.copy()

stemmer = Stemmer()
print(len(set(main_unique_words['words'])))
unique_words['words'] = unique_words['words'].apply(lambda x: stemmer.stem(x))
unique_words = unique_words.drop_duplicates(subset='words').reset_index(drop=True)
print(len(set(unique_words['words'])))


In [ ]:
for i in range(len(category_data_frame['categories'])):
    BOW = [0] * len(unique_words['words'])
    for index, word in enumerate(unique_words['words']):
        if word in category_data_frame['data'][i]:
            count = category_data_frame['data'][i].count(word)
            BOW[index] += count
    category_data_frame['BOW'][i] = BOW

category_data_frame

## now make prediction  

In [ ]:
def predict_with_additive_smooth_with_stemmer():
    sample_data= pd.read_csv('books_test.csv')
    list_of_preds = []
    
    sample_data['description'] = sample_data['description'] + ' ' + sample_data['title']
    sample_data['description'] = sample_data['description'].apply(lambda x: ' '.join([Stemmer().stem(i) for i in word_tokenize(x)]))
    for i in range(450):
         list_of_preds.append([sample_data['categories'][i],get_category_name_by_prediction_with_additive_smooth(sample_data['description'][i])])
    corrects = 0 
    for i in list_of_preds:
        if i[0] == i[1]:
            corrects += 1 
    
    return corrects/450



In [ ]:
predict_result_with_additive_smooth_with_stemmer_res = predict_with_additive_smooth_with_stemmer()
print("percent of correct prediction with additive smooth and step word is : ",predict_result_with_additive_smooth_with_stemmer_res)

## finally get  0.7533333333333333 percent 

## Lemmatizer

In [ ]:
unique_words = main_unique_words.copy()
print(len(unique_words['words']))
lemmetize = Lemmatizer()
unique_words['words'] = unique_words[ 'words' ].apply(lambda x: lemmetize.lemmatize(x))
unique_words = unique_words.drop_duplicates(subset='words').reset_index(drop=True)

print(len(unique_words['words']))

it now has 3488 words for unique words with lemmatizer
now make update that BOW for each category for limmitize words

In [ ]:
for i in range(len(category_data_frame['categories'])):
    BOW = [0] * len(unique_words['words'])
    for index, word in enumerate(unique_words['words']):
        if word in category_data_frame['data'][i]:
            count = category_data_frame['data'][i].count(word)
            BOW[index] += count
    category_data_frame['BOW'][i] = BOW

category_data_frame

In [ ]:
def predict_with_additive_smooth_with_limmitize():
    sample_data= pd.read_csv('books_test.csv')
    list_of_preds = []
    
    sample_data['description'] = sample_data['description'] + ' ' + sample_data['title']
    sample_data['description'] = sample_data['description'].apply(lambda x: ' '.join([lemmetize.lemmatize(i) for i in word_tokenize(x)]))
    for i in range(450):
         list_of_preds.append([sample_data['categories'][i],get_category_name_by_prediction_with_additive_smooth(sample_data['description'][i])])
    corrects = 0 
    for i in list_of_preds:
        if i[0] == i[1]:
            corrects += 1 
    
    return corrects/450



In [ ]:
predict_result_with_additive_smooth_with_limmitize_res = predict_with_additive_smooth_with_limmitize()
print("percent of correct prediction with additive smooth and step word is : ",predict_result_with_additive_smooth_with_limmitize_res)

## finally get 0.8155555555555556 with lemmatizer

by using lemmetize and stemmer we do not have better result than without them
